In [ ]:
from transformers import BertTokenizer,BertModel,get_linear_schedule_with_warmup
from torch.optim import SGD,AdamW
import numpy as np
import pandas as pd
import torch
from torch import nn
import seaborn as sns
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Language Detection.csv')

In [ ]:
text_data=data.iloc[:,0].values
target_data=data.iloc[:,1].values

In [ ]:
target_data

array(['English', 'English', 'English', ..., 'Kannada', 'Kannada',
       'Kannada'], dtype=object)

In [ ]:
Pre_Model='bert-base-multilingual-uncased'
tokenizer=BertTokenizer.from_pretrained(Pre_Model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
text_data

array([' Nature, in the broadest sense, is the natural, physical, material world or universe.',
       '"Nature" can refer to the phenomena of the physical world, and also to life in general.',
       'The study of nature is a large, if not the only, part of science.',
       ...,
       "ಹೇಗೆ ' ನಾರ್ಸಿಸಿಸಮ್ ಈಗ ಮರಿಯನ್ ಅವರಿಗೆ ಸಂಭವಿಸಿದ ಎಲ್ಲವನ್ನೂ ಹೇಳಿದೆ ಮತ್ತು ಅವಳು ಆ ಸಮಯದಿಂದ ತುಂಬಾ ಬದಲಾಗಿದ್ದಾಳೆ.",
       'ಅವಳು ಈಗ ಹೆಚ್ಚು ಚಿನ್ನದ ಬ್ರೆಡ್ ಬಯಸುವುದಿಲ್ಲ ಎಂದು ನಾನು ess ಹಿಸಿದ್ದೇನೆ.',
       'ಟೆರ್ರಿ ನೀವು ನಿಜವಾಗಿಯೂ ಆ ದೇವದೂತನಂತೆ ಸ್ವಲ್ಪ ಕಾಣುತ್ತಿದ್ದೀರಿ ಆದರೆ ನಾನು ಏನು ನೋಡುತ್ತಿದ್ದೇನೆ ನೀವು ಹೇಗೆ ಅವನಾಗಬಹುದು ನೀವು ಇಬ್ಬರು ತುಂಬಾ ಒಳ್ಳೆಯವರು'],
      dtype=object)

In [ ]:
target_data=pd.DataFrame(target_data)

In [ ]:
target_data[0]

,0
0,English
1,English
2,English
3,English
4,English
...,...
10332,Kannada
10333,Kannada
10334,Kannada
10335,Kannada


In [ ]:
target_data[0].unique()

array(['English', 'Malayalam', 'Hindi', 'Tamil', 'Portugeese', 'French',
       'Dutch', 'Spanish', 'Greek', 'Russian', 'Danish', 'Italian',
       'Turkish', 'Sweedish', 'Arabic', 'German', 'Kannada'], dtype=object)

In [ ]:
len(target_data[0].unique())

17

In [ ]:
Language_dict={
    1:'English',
    2:'Malayalam',
    3:'Hindi',
    4:'Tamil',
    5:'Portugeese',
    6:'French',
    7:'Dutch',
    8:'Spanish',
    9:'Greek',
    10:'Russian',
    11:'Danish',
    12:'Italian',
    13:'Turkish',
    14:'Sweedish',
    15:'Arabic',
    16:'German',
    17:'Kannada'
}

In [ ]:
#from sklearn.compose import ColumnTransformer
#from sklearn.preprocessing import OneHotEncoder
#ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
#target_data= np.array(ct.fit_transform(target_data))

In [ ]:
target_data.replace('English',1,inplace=True)
target_data.replace('Malayalam',2,inplace=True)
target_data.replace('Hindi',3,inplace=True)
target_data.replace('Tamil',4,inplace=True)
target_data.replace('Portugeese',5,inplace=True)
target_data.replace('French',6,inplace=True)
target_data.replace('Dutch',7,inplace=True)
target_data.replace('Spanish',8,inplace=True)
target_data.replace('Greek',9,inplace=True)
target_data.replace('Russian',10,inplace=True)
target_data.replace('Danish',11,inplace=True)
target_data.replace('Italian',12,inplace=True)
target_data.replace('Turkish',13,inplace=True)
target_data.replace('Sweedish',14,inplace=True)
target_data.replace('Arabic',15,inplace=True)
target_data.replace('German',16,inplace=True)
target_data.replace('Kannada',17,inplace=True)

In [ ]:
target_data

,0
0,1
1,1
2,1
3,1
4,1
...,...
10332,17
10333,17
10334,17
10335,17


In [ ]:
from sklearn.model_selection import train_test_split
train_text_data,test_text_data,train_target_data,test_target_data=train_test_split(text_data,target_data,shuffle=True,test_size=0.3)

In [ ]:
test_target_data

,0
10039,17
4764,7
3348,6
995,1
7150,12
...,...
8186,13
1239,1
2891,5
9519,16


In [ ]:
test_text_data

array(['ನುಡಿಗಟ್ಟು ನಂಬರ್ ಒನ್ ಇದು ಕೇಳಲು ಇನ್ನೊಂದು ಮಾರ್ಗ.',
       'en je weet dat het beste ervan was toen ik mijn trouwjurk droeg die ook van goud was.',
       "Évolutions sémantiques et esthétiques Ce changement de représentation se fit à la faveur d'un changement linguistique majeur\xa0: l'apparition du français classique[20].",
       ...,
       'Outras razões pelas quais esta mensagem pode aparecer: \n.estamos todos procurando maneiras rápidas de nos tornarmos mais fluentes em inglês, eu sei que entendo o que acontece é que existem apenas algumas palavras circulando em nosso cérebro e precisamos fazer toda essa conversa usando apenas incrível incrível legal ou épico, então que tal nós aprenda algumas palavras novas e inteligentes em inglês que você pode usar em sua vida diária e elevar seu vocabulário em minutos.',
       'Geld, aber Sie würden diesen Bankkredit nicht erhalten, wenn Sie der Bank nicht beweisen, dass Sie genug Geld haben, damit Sie sich das nächste Mal in einer schw

In [ ]:
#token_lens_hp=[]
#for txt in train_text_data:
    #tokens=tokenizer.encode(txt,max_length=512)
    #token_lens_hp.append(len(tokens))
#sns.distplot(token_lens_hp)
#plt.xlim([0, 256]);
#plt.xlabel('Token count');

In [ ]:
max_len=150

In [ ]:
class preprocess(Dataset):
    def __init__(self, text, labels, tokenizer, max_len):
        self.text = text
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        text = str(self.text[item])
        labels = self.labels.iloc[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

In [ ]:
def Data_loader(train_data_np,target_data_np,tokenizer,max_len,batch_size):
    ds=preprocess(train_data_np,target_data_np,tokenizer,max_len)
    return DataLoader(ds,batch_size=batch_size,num_workers=2)

In [ ]:
batch_size=32
train_dataloader=Data_loader(train_text_data,train_target_data,tokenizer,max_len,batch_size)
test_dataloader=Data_loader(test_text_data,test_target_data,tokenizer,max_len,batch_size)

In [ ]:
train_text_data.shape

(7235,)

In [ ]:
data=next(iter(train_dataloader))
data.keys()
data['labels'].shape

data['labels'].squeeze

<function Tensor.squeeze>

In [ ]:
data['labels'].shape

torch.Size([32, 1])

In [ ]:
data['labels']

tensor([[14],
        [12],
        [ 5],
        [14],
        [ 9],
        [17],
        [ 4],
        [ 6],
        [11],
        [15],
        [15],
        [ 6],
        [10],
        [ 7],
        [ 8],
        [ 3],
        [ 6],
        [ 6],
        [ 7],
        [ 5],
        [10],
        [ 5],
        [ 6],
        [ 6],
        [10],
        [ 1],
        [ 6],
        [15],
        [ 4],
        [10],
        [12],
        [12]])

In [ ]:
class classifier(nn.Module):
    def __init__(self,n_classes):
        super(classifier,self).__init__()
        self.bert=BertModel.from_pretrained(Pre_Model,return_dict=False)
        self.dropout=nn.Dropout(0.4)
        self.out=nn.Linear(self.bert.config.hidden_size,n_classes+1)
        self.num_classes = n_classes
    def forward(self,input_ids,attention_mask):
        _,pooled_output=self.bert(input_ids=input_ids,attention_mask=attention_mask)
        output=self.dropout(pooled_output)
        return self.out(output)

In [ ]:
classifier(len(Language_dict))

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

classifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
model=classifier(len(Language_dict))
model=model.to(device)

In [ ]:
EPOCHS=6
optimizer=AdamW(model.parameters(),lr=2e-5)
total_steps=len(train_dataloader)*EPOCHS
scheduler=get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=total_steps)
loss_fn=nn.CrossEntropyLoss().to(device)

In [ ]:
def train_epochs(model,data_loader,loss_fn,optimizer,device,scheduler,n_examples):
    model=model.train()
    total_loss=0
    num_batches=0
    for d in data_loader:
        input_ids=d['input_ids'].to(device)
        attention_mask=d['attention_mask'].to(device)
        labels=d['labels'].to(device).squeeze()
        outputs=model(input_ids=input_ids,attention_mask=attention_mask)
        _,preds=torch.max(outputs,dim=1)
        loss=loss_fn(outputs,labels)
        num_batches+=1
        total_loss+=loss.item()
        average_loss=total_loss/num_batches
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(),max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    return average_loss

In [ ]:
for epoch in range(EPOCHS):
    print(f'Epoch{epoch+1}/{EPOCHS}')
    print('-'*10)
    train_loss=train_epochs(model,train_dataloader,loss_fn,optimizer,device,scheduler,len(train_text_data))
    print(f'train loss{train_loss}')

Epoch1/6
----------
train loss0.7046038866010269
Epoch2/6
----------
train loss0.059921327473735235
Epoch3/6
----------
train loss0.0276140404684625
Epoch4/6
----------
train loss0.017042447115389
Epoch5/6
----------
train loss0.013526398094682769
Epoch6/6
----------
train loss0.009920498631106861


In [ ]:
test_text='आप कहां जा रहे हैं'

In [ ]:
model.eval()
encoding = tokenizer.encode_plus(
            test_text,
            add_special_tokens=True,
            max_length=len(test_text),
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
input_ids=encoding['input_ids'].to(device)
attention_mask=encoding['attention_mask'].to(device)
outputs=model(input_ids=input_ids,attention_mask=attention_mask)
_,preds=torch.max(outputs,dim=1)
preds.item()
Language_dict[preds.item()]

'Hindi'